In [ ]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [ ]:
df = pd.read_csv('Sources/fiuba_6_avisos_detalle_concat.csv').drop(['Unnamed: 0'],axis=1)
cluster = pd.read_csv('Sources/idaviso-cluster.csv').drop('Unnamed: 0',axis=1)
features = pd.read_csv('Sources/mas_features.csv').drop('Unnamed: 0',axis=1)
tuples = [tuple(x) for x in df.values]
df.head()

In [ ]:
for x in tuples:
    x = tuple(x)

In [ ]:
avisos = sc.parallelize(tuples)

In [ ]:
avisos = avisos.map(lambda x: (x[0],x[1],x[2],str.upper(re.sub("<.*?>", " ", x[3])),str.upper(re.sub("<.*?>", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10])).map(lambda x: (x[0],x[1],x[2],str(re.sub("\xa0", " ", x[3])),str(re.sub("\xa0", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10]))

In [ ]:
import re, string, unicodedata
def removePunctuation(text):
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

In [ ]:
def palabrasLargas(texto):
    lista = texto.split(' ') 
    lista = [x for x in lista if not len(x)>17]
    return " ".join(lista)

In [ ]:
avisos = avisos.map((lambda x: (x[0],x[1],x[2],removePunctuation(x[3]),removePunctuation(x[4]),x[5],x[6],x[7],x[8],x[9],x[10]))).map((lambda x: (x[0],x[1],x[2],palabrasLargas(x[3]),x[4],x[5],x[6],x[7],x[8], x[9],x[10]))).collect()

In [ ]:
avisos = pd.DataFrame.from_records(avisos)
avisos.head()

In [ ]:
for x in avisos.columns:  
    avisos = avisos.rename(columns = {x:df.columns[x]})
avisos.head()
avisos = avisos.drop(['idpais','nombre_zona','mapacalle', 'ciudad'], axis=1)
avisos.head()

In [ ]:
avisos = avisos.merge(cluster,on='idaviso',how='left')
avisos.head()

In [ ]:
avisos = avisos.merge(features, on='idaviso',how='left')
avisos.head()

In [ ]:
des = avisos['descripcion'].tolist()
idaviso = avisos['idaviso'].tolist()

In [ ]:
x = []
for i in range (len(des)):
    x.append((idaviso[i],des[i]))

In [ ]:
des = sc.parallelize(x)
des.map(lambda x: (x[0],x[1]))

In [ ]:
import re, string, unicodedata
def removePunctuation(text):    
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

des = des.map(lambda x:(x[0],removePunctuation(x[1])))

In [ ]:
stopwords = ['el', 'la', 'los', 'les', 'las', 'de', 'del', 'a', 'ante', 'con', 'en', 'para', 'por', 'y', 'o', 'u', 'tu', 'te', 'ti', 'le', 'que', 'al', 'ha', 'un', 'han', 'lo', 'su', 'una', 'e','estas', 'esto', 'este', 'es', 'tras', 'suya', 'a', 'acá', 'ahí', 'ajena', 'ajenas', 'ajeno', 'ajenos', 'al', 'algo', 'algún', 'alguna', 'algunas', 'alguno', 'algunos', 'allá', 'alli', 'allí', 'ambos', 'ampleamos', 'ante', 'antes', 'aquel', 'aquella', 'aquellas', 'aquello', 'aquellos', 'aqui', 'aquí', 'arriba', '', 'asi', 'atras', 'aun', 'aunque', 'bajo', 'bastante', 'bien', 'cabe', 'cada', 'casi', 'cierta', 'ciertas', 'cierto', 'ciertos', 'como', 'cómo', 'con', 'conmigo', 'conseguimos', 'conseguir', 'consigo', 'consigue', 'consiguen', 'consigues', 'contigo', 'contra', 'cual', 'cuales', 'cualquier', 'cualquiera', 'cualquieras', 'cuancuán', 'cuando', 'cuanta', 'cuánta', 'cuantas', 'cuántas', 'cuanto', 'cuánto', 'cuantos', 'cuántos', 'de', 'dejar', 'del', 'demás', 'demas', 'demasiada', 'demasiadas', 'demasiado', 'demasiados', '', 'dentro', 'desde', 'donde', 'dos', 'el', 'él', 'ella', 'ellas', '', 'ello', 'ellos', 'empleais', 'emplean', 'emplear', 'empleas', 'empleo', 'en', 'encima', 'entonces', 'entre', 'era', 'eramos', 'eran', 'eras', 'eres', 'es', 'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estaba', 'estado', 'estais', 'estamos', 'estan', 'estar', 'estas', 'este', 'esto', 'estos', 'estoy', 'etc', 'fin', 'fue', 'fueron', 'fui', 'fuimos', 'gueno', 'ha', 'hace', 'haceis', 'hacemos', 'hacen', 'hacer', 'haces', 'haciahaago', 'hasta', 'incluso', 'intenta', 'intentais', 'intentamos', 'intentan', 'intentar', 'intentas', 'intento', 'ir', 'jamás', 'junto', 'juntos', 'la', 'largo', 'las', 'lo', 'los', 'mas', 'más', 'me', 'menos', 'mi', 'mía', 'mia', 'mias', 'mientras', 'mio', 'mío', 'mios', 'mis', 'misma', 'mismas', 'mismo', 'mismos', 'modo', 'mucha', 'muchas', 'muchísima', 'muchísimas', 'muchísimo', 'muchísimos', 'mucho', 'muchos', 'muy', 'nada', 'ni', 'ningun', 'ninguna', 'ningunas', 'ninguno', 'ningunos', 'no', 'nos', 'nosotras', 'nosotros', 'nuestra', 'nuestras', 'nuestro', 'nuestros', 'nunca', 'os', 'otra', 'otras', 'otro', 'otros', 'para', 'parecer', 'pero', 'poca', 'pocas', 'poco', 'pocos', 'podeis', 'podemos', 'poder', 'podria', 'podriais', 'podriamos', 'podrian', 'podrias', 'por', 'por', 'qué', 'porque', 'primero', 'primero', 'desde', 'puede', 'pueden', 'puedo', 'pues', 'que', 'qué', 'querer', 'quien', 'quién', 'quienes', 'quienes', 'quiera', 'quienquiera', 'quiza', 'quizas', 'sabe', 'sabeis', 'sabemos', 'saben', 'saber', 'sabes', 'se', 'segun', 'ser', 'si', 'sí', 'siempre', 'siendo', 'sin', 'sín', 'sino', 'so', 'sobre', 'sois', 'solamente', 'solo', 'somos', 'soy', 'sr', 'sra', 'sres', 'esta', 'su', 'sus', 'suya', 'suyas', 'suyo', 'suyos', 'tal', 'tales', 'también', 'tambien', 'tampoco', 'tan', 'tanta', 'tantas', 'tanto', 'tantos', 'te', 'teneis', 'tenemos', 'tener', 'tengo', 'ti', 'tiempo', 'tiene', 'tienen', 'toda', 'todas', 'todo', 'todos', 'tomar', 'trabajatrabajais', 'trabajamos', 'trabajan', 'trabajar', 'trabajas', 'trabajo', 'tras', 'tú', 'tu', 'tus', 'tuya', 'tuyo', 'tuyos', 'ultimo', 'un', 'una', 'unas', 'uno', 'unos', 'usa', 'usais', 'usamos', 'usan', 'usar', 'usas', 'uso', 'usted', 'ustedes', 'va', 'vais', 'valor', 'vamos', 'van', 'varias', 'varios', 'vaya', 'verdad', 'verdadera', 'vosotras', 'vosotros', 'voy', 'vuestra', 'vuestras', 'vuestro', 'vuestros', 'y', 'ya', 'yo', 'como', 'cómo', 'hacer', 'se', 'tengo']
def removestopwords(x):
    return not(x in stopwords)

In [ ]:
def list_features(x):
    one_hot=[0,0,0,0,0,0,0]
    palabras=['sabados','domingos','ingles','3 anos','2 anos','4 anos', '5 anos']
    for palabra in palabras:
        if palabra in x:
            
            one_hot[palabras.index(palabra)] = 1
    return one_hot            

x = des.map(lambda x: (x[0],list_features(x[1]))).map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3],x[1][4],x[1][5],x[1][6])).collect()

In [ ]:
df = pd.DataFrame.from_records(x)

In [ ]:
df.columns=['idaviso','sabados','domingos','ingles','3 anos','2 anos','4 anos', '5 anos']

In [ ]:
avisos = avisos.merge(df, on='idaviso',how='left')

In [ ]:
avisos.to_csv("Sources/fiuba_6_avisos_detalle_limpio.csv")